In [19]:
import cv2
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

class DataSchema(object):
    def __init__(self, schema):
        self._idx2name = schema.split(',')
        self._name2idx = dict(((x, y) for (y, x) in enumerate(schema)))
    
    def name(idx):
        return self._ind2name[idx]
    
    def idx(name):
        return self._name2idx[name]

STR_SCHEMA = 'SteerAngle,Throttle,Brake,Speed,X_Position,Y_Position,Pitch,Yaw,Roll'
SCHEMA = DataSchema(STR_SCHEMA)
FRAME_SHAPE = (160, 320)
DST_SIZE = 5
BOTTOM_OFFSET = 6

SOURCE = np.float32([[14.32 , 140.71], [ 120.78, 95.5], [199.49 ,96.84], [302.7 ,140.71]])

TARGET = np.float32([[FRAME_SHAPE[1]/2 - DST_SIZE, 
                      FRAME_SHAPE[0] - BOTTOM_OFFSET],
                  [FRAME_SHAPE[1]/2 - DST_SIZE, 
                   FRAME_SHAPE[0] - BOTTOM_OFFSET - 2 * DST_SIZE],
                  [FRAME_SHAPE[1]/2 + DST_SIZE, 
                   FRAME_SHAPE[0] - BOTTOM_OFFSET - 2 * DST_SIZE], 
                  [FRAME_SHAPE[1]/2 + DST_SIZE, 
                   FRAME_SHAPE[0] - BOTTOM_OFFSET]])

FRAME_ORIGIN = (FRAME_SHAPE[1]/2, FRAME_SHAPE[0])

def rover_coords(binary_img):
    # Extract xpos and ypos pixel positions from binary_img and
    # Convert xpos and ypos to rover-centric coordinates
    ypos, xpos = binary_img.nonzero()
    xpos = xpos.astype('float32')
    ypos = ypos.astype('float32')
    nxpos = FRAME_ORIGIN[1] - ypos
    nypos = FRAME_ORIGIN[0] - xpos
    return nxpos, nypos

In [23]:
def load_dataset(path):    
    with open(path) as ip:
        lines = list(ip)
    lines = filter(lambda x: STR_SCHEMA not in x, lines)
    rows = [x.split(',') for x in lines]
    images = [x[0] for x in rows]
    features = [[float(x) for x in row[1:]] for row in rows]
    features = np.array(features, dtype=np.float32)
    
    all_images = None
    for index, img in enumerate(images):
        image = mpimg.imread(img)
        if all_images is None:
            all_images = np.zeros((len(images),) + image.shape, dtype=image.dtype)
        all_images[index] = image
    return features, all_images

In [24]:
features, images = load_dataset('./recorder/2017_MAY_27/robot_log.csv')

In [ ]:
def color_thresh(img, rgb_thresh=(0, 0, 0)):
    ####### TODO 
    # Create an empty array the same size in x and y as the image 
        # but just a single channel
    # Apply the thresholds for RGB and 
        # assign 1's where threshold was exceeded
    thresh = np.array(rgb_thresh)
    thresh.reshape((1, 1, -1))
    binary_image = img > thresh
    binary_image = np.all(binary_image, axis=-1)
    binary_image = binary_image.astype('uint8')
    return binary_image

def perspect_transform(img, src, dst):

    # Get transform matrix using cv2.getPerspectivTransform()
    M = cv2.getPerspectiveTransform(src, dst)
    # Warp image using cv2.warpPerspective()
    # keep same size as input image
    warped = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]))
    # Return the result
    return warped